# Guía de uso: Docker y Kubernetes (teoría + comandos)

Este cuaderno sirve como guía para:

- Entender qué son **Docker** y **Kubernetes** y en qué casos se usan.
- Preparar un entorno de trabajo aislado en Python.
- Conocer y practicar los **comandos básicos de Docker** y la gestión de contenedores.
- Conocer y practicar los **comandos básicos de Kubernetes**, incluyendo namespaces, pods, deployments y services.
- Ver cómo se conectan Docker y Kubernetes, tanto a nivel teórico como mediante ejemplos de configuración.

Requisitos previos instalados en la máquina (fuera de Python):

- `python` (3.9+ recomendado)
- `pip`
- `docker`
- `kubectl`
- Un clúster de Kubernetes (por ejemplo `minikube` o `kind`).


## Índice

1. [Introducción a Docker y Kubernetes](#sec1)
2. [Entorno de trabajo en Python y `requirements.txt`](#sec2)
3. [Docker: conceptos, comandos y configuración de contenedores](#sec3)
4. [Kubernetes: conceptos, comandos y namespaces](#sec4)
5. [Conexión entre Docker y Kubernetes](#sec5)


<a id="sec1"></a>
## 1. Introducción a Docker y Kubernetes

### 1.1. Docker

Docker es una plataforma para empaquetar y ejecutar aplicaciones en **contenedores**.

Conceptos clave:

- **Imagen**: plantilla inmutable que incluye sistema de archivos, dependencias y configuración necesaria para ejecutar una aplicación.
- **Contenedor**: instancia en ejecución de una imagen; es un proceso aislado con su propio sistema de archivos, red y configuración.
- **Dockerfile**: archivo de texto que describe cómo construir una imagen (capas, comandos, configuración).
- **Registro de contenedores**: repositorio donde se almacenan y distribuyen imágenes (Docker Hub, GitHub Container Registry, registros privados, etc.).

Usos típicos de Docker:

- Crear entornos de desarrollo reproducibles.
- Empaquetar microservicios.
- Integrar aplicaciones en pipelines de CI/CD.
- Simular entornos de producción en local.


### 1.2. Kubernetes

Kubernetes (K8s) es un **orquestador de contenedores**. Su objetivo es ejecutar y gestionar aplicaciones basadas en contenedores en un cluster de nodos.

Conceptos clave:

- **Cluster**: conjunto de nodos gestionados por Kubernetes.
- **Nodo**: máquina (física o virtual) en la que se ejecutan pods.
- **Pod**: unidad mínima de despliegue en Kubernetes; contiene uno o varios contenedores que comparten red y almacenamiento.
- **Deployment**: recurso que gestiona un conjunto de réplicas de pods, sus actualizaciones y su estado deseado.
- **Service**: recurso que expone uno o varios pods a través de una dirección IP y puerto estables.
- **Namespace**: separa lógicamente recursos dentro del cluster (por ejemplo, separación por entorno: `dev`, `test`, `prod`).

Usos típicos de Kubernetes:

- Desplegar aplicaciones a gran escala.
- Escalar horizontalmente (añadir/eliminar réplicas) según la carga.
- Asegurar alta disponibilidad y auto-recuperación.
- Coordinar actualizaciones (rolling updates, rollbacks).


### 1.3. Cómo se relacionan Docker y Kubernetes

- Docker proporciona el formato de **imagen** y la tecnología de **contenedores**.
- Kubernetes orquesta **muchos contenedores** en uno o varios nodos.

Flujo típico de trabajo:

1. La aplicación se empaqueta en una **imagen Docker**.
2. La imagen se sube a un **registro de contenedores**.
3. Kubernetes descarga la imagen desde el registro.
4. Kubernetes crea **pods** que ejecutan contenedores basados en esa imagen.
5. Kubernetes se encarga del **escalado**, reinicio automático, distribución de carga y exposición mediante **Services**.


<a id="sec2"></a>
## 2. Entorno de trabajo en Python y gestión de dependencias

En esta sección se explica cómo preparar un entorno de trabajo aislado en Python y cómo generar un archivo `requirements.txt` en cualquier momento a partir de las dependencias instaladas.


### 2.1. Crear y activar un entorno virtual

Un **entorno virtual** (virtualenv) permite aislar las librerías de un proyecto del resto del sistema.

Ejecutar estos comandos en la terminal (fuera de Jupyter):


In [ ]:
# Comandos de terminal (no se ejecutan en esta celda, se copian a la shell)

# Crear carpeta de proyecto
mkdir proyecto-docker-k8s
cd proyecto-docker-k8s

# Crear entorno virtual
python -m venv .venv

# Activar entorno virtual
# En Linux/macOS:
#   source .venv/bin/activate
# En Windows (PowerShell):
#   .venv\Scripts\Activate.ps1

# Verificar que pip usa el entorno virtual
python -m pip --version

### 2.2. Instalación manual de dependencias

Con el entorno virtual activado se instalan las dependencias una a una usando `pip install`.

Dependencias principales:

- `docker`: cliente de Python para interactuar con el daemon de Docker.
- `kubernetes`: cliente de Python para interactuar con la API de Kubernetes.
- `python-dotenv`: carga variables de entorno desde archivos `.env` (útil para configuración).

Dependencias opcionales:

- `requests`: para hacer peticiones HTTP.
- `pyyaml`: para leer y escribir archivos YAML desde Python.


In [ ]:
# Ejecutar en la terminal con el entorno virtual activado

# Dependencias principales
pip install docker
pip install kubernetes
pip install python-dotenv

# Dependencias opcionales
pip install requests
pip install pyyaml

### 2.3. Generar `requirements.txt` en cualquier momento

Una vez instaladas las dependencias (o después de modificar el entorno), se puede generar el archivo `requirements.txt` automáticamente con:

```bash
pip freeze > requirements.txt
```

Este archivo contendrá todas las librerías instaladas en el entorno virtual junto con sus versiones exactas.

Para ver su contenido:

```bash
cat requirements.txt
```

Este proceso se puede repetir siempre que se añadan o eliminen dependencias.


<a id="sec3"></a>
## 3. Docker: conceptos, comandos y configuración de contenedores

En esta sección se revisan los comandos más importantes de Docker y cómo configurar el comportamiento interno de un contenedor.


### 3.1. Comandos básicos de información

Estos comandos proporcionan información general sobre Docker, las imágenes locales y los contenedores.


In [ ]:
# Información general de Docker (cliente y servidor)
!docker version
!docker info

# Imágenes locales disponibles
!docker images

# Contenedores en ejecución
!docker ps

# Todos los contenedores (incluidos detenidos)
!docker ps -a

### 3.2. Creación y ejecución de contenedores

El comando principal es `docker run`, que crea y arranca un contenedor a partir de una imagen.

Opciones frecuentes:

- `-d`: ejecuta el contenedor en segundo plano (modo detached).
- `--name`: asigna un nombre al contenedor.
- `-p host:contenedor`: mapea un puerto del host al puerto del contenedor.
- `-e CLAVE=valor`: define variables de entorno.
- `-v ruta_host:ruta_contenedor`: monta un volumen.
- `--memory` y `--cpus`: limitan recursos de memoria y CPU.


In [ ]:
# Ejecutar un contenedor de nginx en segundo plano, accesible en http://localhost:8080
!docker run -d --name web-nginx -p 8080:80 nginx

# Comprobar que el contenedor está en ejecución
!docker ps

### 3.3. Configuración de recursos y entorno

#### 3.3.1. Limitación de memoria y CPU

Permite controlar el uso de recursos de cada contenedor:

```bash
docker run -d --name web-limitada \
  --memory="256m" \
  --cpus="1.0" \
  -p 8081:80 nginx
```

#### 3.3.2. Variables de entorno

Las variables de entorno son útiles para configurar el comportamiento de la aplicación dentro del contenedor:

```bash
docker run -d --name app-env \
  -e APP_ENV=production \
  -e DEBUG=false \
  nginx
```

#### 3.3.3. Volúmenes y sistema de archivos

Los volúmenes permiten persistir datos y compartir archivos entre el host y el contenedor:

```bash
docker run -d --name web-volumen \
  -v /ruta/local/html:/usr/share/nginx/html \
  -p 8082:80 nginx
```


### 3.4. Configuración del proceso y comando de arranque

En un `Dockerfile` se pueden definir:

- `CMD`: comando por defecto al iniciar el contenedor.
- `ENTRYPOINT`: ejecutable principal; `CMD` se puede usar como argumentos adicionales.
- `WORKDIR`: directorio de trabajo.

Ejemplo de `Dockerfile` genérico:

```dockerfile
FROM python:3.11-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

ENV APP_ENV=production

EXPOSE 5000

CMD ["python", "app.py"]
```

Construcción y ejecución:

```bash
docker build -t miapp:latest .
docker run -d --name miapp -p 5000:5000 miapp:latest
```


### 3.5. Operaciones dentro del contenedor

#### 3.5.1. Acceder a una shell

```bash
docker exec -it web-nginx /bin/bash
```

#### 3.5.2. Ver logs del contenedor

```bash
docker logs web-nginx
docker logs -f web-nginx   # seguir logs en tiempo real
```

#### 3.5.3. Copiar archivos

```bash
docker cp archivo.txt web-nginx:/tmp/
docker cp web-nginx:/etc/nginx/nginx.conf ./nginx.conf
```


### 3.6. Gestión y limpieza de contenedores e imágenes

#### Detener y eliminar contenedores

```bash
docker stop web-nginx web-limitada web-volumen app-env miapp
docker rm web-nginx web-limitada web-volumen app-env miapp
```

#### Eliminar imágenes

```bash
docker images
docker rmi nginx miapp:latest
```

#### Limpieza avanzada

```bash
docker container prune -f    # elimina contenedores detenidos
docker image prune -a -f     # elimina imágenes no utilizadas
```


<a id="sec4"></a>
## 4. Kubernetes: conceptos, comandos y namespaces

En esta sección se revisan los comandos principales de `kubectl` y la configuración de recursos básicos de Kubernetes.


### 4.1. Conceptos y comandos básicos

#### Información general del cluster

```bash
kubectl cluster-info
kubectl get nodes
kubectl get all --all-namespaces
```

#### Contextos de Kubernetes

```bash
kubectl config get-contexts
kubectl config current-context
```


### 4.2. Namespaces

Un **namespace** agrupa recursos en un espacio lógico separado.

#### Comandos básicos de namespaces

```bash
kubectl get namespaces
kubectl create namespace demo
kubectl describe namespace demo
kubectl delete namespace demo
```

#### Establecer un namespace por defecto en el contexto actual

```bash
kubectl config set-context --current --namespace=demo
```


### 4.3. Pods: creación y gestión

#### Crear un pod simple con `kubectl run`

```bash
kubectl run miweb --image=nginx
```

#### Ver y describir pods

```bash
kubectl get pods
kubectl get pods -o wide
kubectl describe pod miweb
```

#### Logs y comandos dentro del pod

```bash
kubectl logs miweb
kubectl exec -it miweb -- /bin/bash
```


### 4.4. Deployments: gestión de réplicas y actualizaciones

Un **Deployment** gestiona un conjunto de réplicas de pods y define el estado deseado de la aplicación.

#### Crear un Deployment básico

```bash
kubectl create deployment miweb --image=nginx
```

#### Ver y describir Deployments

```bash
kubectl get deployments
kubectl describe deployment miweb
```

#### Escalar réplicas

```bash
kubectl scale deployment miweb --replicas=3
kubectl get pods -o wide
```

#### Actualizar la imagen utilizada por un Deployment

```bash
kubectl set image deployment/miweb nginx=nginx:latest
```


### 4.5. Services: exposición de aplicaciones

Un **Service** proporciona una IP y un puerto estables para acceder a uno o varios pods.

Tipos habituales:

- `ClusterIP`: accesible solo dentro del cluster.
- `NodePort`: expone un puerto en cada nodo para acceso externo.
- `LoadBalancer`: integra con balanceadores de carga externos (principalmente en nubes públicas).

#### Exponer un Deployment como Service

```bash
kubectl expose deployment miweb --port=80 --type=NodePort
```

#### Ver Services

```bash
kubectl get svc
kubectl describe svc miweb
```


### 4.6. Definición de recursos mediante manifiestos YAML

Aunque se pueden crear recursos con comandos, lo habitual es definirlos en archivos YAML.

#### Ejemplo de Deployment con configuración adicional

El siguiente manifiesto incluye:

- Réplicas.
- Labels.
- Variables de entorno.
- Peticiones y límites de recursos.
- Probes de readiness y liveness.


In [ ]:
%%writefile deployment-ejemplo.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: miapp
  labels:
    app: miapp
spec:
  replicas: 2
  selector:
    matchLabels:
      app: miapp
  strategy:
    type: RollingUpdate
    rollingUpdate:
      maxUnavailable: 1
      maxSurge: 1
  template:
    metadata:
      labels:
        app: miapp
        tier: backend
    spec:
      containers:
        - name: miapp
          image: usuario/miapp:latest
          ports:
            - containerPort: 5000
          env:
            - name: APP_ENV
              value: "production"
          resources:
            requests:
              cpu: "100m"
              memory: "128Mi"
            limits:
              cpu: "500m"
              memory: "256Mi"
          readinessProbe:
            httpGet:
              path: /
              port: 5000
            initialDelaySeconds: 5
            periodSeconds: 10
          livenessProbe:
            httpGet:
              path: /
              port: 5000
            initialDelaySeconds: 15
            periodSeconds: 20

In [ ]:
# Aplicar el Deployment definido en YAML
!kubectl apply -f deployment-ejemplo.yaml

# Ver Deployments y pods asociados
!kubectl get deployments
!kubectl get pods -l app=miapp -o wide

#### Ejemplo de Service asociado

Service de tipo `NodePort` que expone la aplicación en el puerto 30080 de los nodos del cluster.


In [ ]:
%%writefile service-ejemplo.yaml
apiVersion: v1
kind: Service
metadata:
  name: miapp-svc
spec:
  selector:
    app: miapp
  type: NodePort
  ports:
    - name: http
      port: 5000
      targetPort: 5000
      nodePort: 30080

In [ ]:
# Aplicar el Service definido en YAML
!kubectl apply -f service-ejemplo.yaml

# Ver Services
!kubectl get svc

<a id="sec5"></a>
## 5. Conexión entre Docker y Kubernetes

### 5.1. Flujo de trabajo completo

1. **Desarrollar la aplicación** (por ejemplo, en Python).
2. **Empaquetar en una imagen Docker** mediante un `Dockerfile`.
3. **Probar la imagen localmente** con `docker run`.
4. **Publicar la imagen** en un registro (Docker Hub u otro).
5. **Definir manifiestos de Kubernetes** (`Deployment`, `Service`, etc.) que referencian la imagen.
6. **Aplicar los manifiestos** en el cluster con `kubectl apply`.
7. **Gestionar escalado, actualizaciones y supervisión** con `kubectl` y las primitivas de Kubernetes.


### 5.2. Ejemplo de `Dockerfile` genérico

```dockerfile
FROM python:3.11-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

ENV APP_ENV=production

EXPOSE 5000

CMD ["python", "app.py"]
```

Comandos para construir y probar la imagen:

```bash
docker build -t usuario/miapp:latest .
docker run -d --name miapp -p 5000:5000 usuario/miapp:latest
```


### 5.3. Ejemplo de Deployment y Service que usan la imagen Docker

Una vez publicada la imagen en un registro accesible por el cluster, los manifiestos de Kubernetes referencian esa imagen.

#### Deployment

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: miapp
spec:
  replicas: 2
  selector:
    matchLabels:
      app: miapp
  template:
    metadata:
      labels:
        app: miapp
    spec:
      containers:
        - name: miapp
          image: usuario/miapp:latest
          ports:
            - containerPort: 5000
```

#### Service

```yaml
apiVersion: v1
kind: Service
metadata:
  name: miapp-svc
spec:
  selector:
    app: miapp
  type: NodePort
  ports:
    - port: 5000
      targetPort: 5000
      nodePort: 30080
```

Aplicación al cluster:

```bash
kubectl apply -f deployment-miapp.yaml
kubectl apply -f service-miapp.yaml
kubectl get pods
kubectl get svc
```

Con estos pasos se ve claramente cómo Docker y Kubernetes se integran: Docker crea las imágenes y Kubernetes las usa como base para orquestar los contenedores a gran escala.
